# Monarch Data Processing Notebook
## Import the dependencies

In [ ]:
import asyncio
from anyio import get_cancelled_exc_class
from monarchmoney import MonarchMoney
mm = MonarchMoney()
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import datetime
import plotly.express as px
import plotly.io as pio

In [ ]:
# Load the Monarch Money session token.  If it doesn't exist, prompt the user to login

try:
    mm.load_session(filename="/data/mm_session.pickle")
    print("Monarch session has been loaded successfully")
except FileNotFoundError as e:
    # Session not found, login on the console.
    print("A console should appear.  In the console, enter \"await mm.interactive_login()\" then rerun the notebook.")
    %qtconsole

## Define key constants
Change these values to specify:
* Date range of transactions to download
* List of tags to exclude

In [ ]:
# Specify the start and end dates (inclusive) in YYYY, MM, DD format
# Dates must be at least 3 days apart
startDate = datetime.datetime(2023, 1, 1)
endDate   = datetime.datetime(2024, 12, 31)

excludeTagsList = ['Wedding']


## Get the data from Monarch
This cell downloads all the transactions for the specified dates (inclusive).
If there are too many transactions, the request will time out, so this cell will recursively divide the date range until the number of transactions can successfully be downloaded within the time limit.

In [ ]:
# Concatenate all the trancactions from startDate to endDate
# This approach allows you to get transactions for arbitrary date ranges
# while avoiding TimeoutErrors


# Assumption: startDate and endDate are >3 days apart at a minimum
#
# Params:
#   transactions - dataframe of aggregate transactions
#   startDate    - datetime object of the start date
#   endDate      - datetime object of the end date
# Returns:
#   dataframe of aggregate transaction data
async def getTransactions(transactions, startDate, endDate):
    # Check if the dates are within 2 days.  If the API calls are failing enough to reduce the window
    # to 2 days, something is likely very wrong.
    if ((endDate - startDate <= datetime.timedelta(days=2))):
        print("Unable to successfully obtain transaction data")
        raise UserWarning("Program Error")
    
    # Try to get the transaction data
    try:
        t = await mm.get_transactions(limit = 5000, 
                                      start_date = startDate.strftime('%Y-%m-%d'), 
                                      end_date = endDate.strftime('%Y-%m-%d'))
        return pd.concat([ transactions, pd.DataFrame(t['allTransactions']['results']) ])
    except (get_cancelled_exc_class(), TimeoutError) as e:
        # If we get here it's because of a timeout error, likely due to there being too many transactions to download.
        # Since the upstream libraries don't offer a method to increase the timeout period, our workaround is to batch the
        # API calls to request a smaller number of transactions in each call.  Recurse until we have success.
        delta = datetime.timedelta( ((endDate - startDate) / 2).days)
        t1_end = (endDate - delta - datetime.timedelta(seconds=1))
        t2_start = t1_end + datetime.timedelta(days=1)
        print("Timeout, trying " + startDate.strftime('%Y-%m-%d') + " to " + t1_end.strftime('%Y-%m-%d') + 
              " and " + t2_start.strftime('%Y-%m-%d') + " to " + endDate.strftime('%Y-%m-%d'))
        t1 = await getTransactions(transactions, startDate, t1_end)
        t2 = await getTransactions(transactions, t2_start, endDate)
        return pd.concat([ transactions, t1, t2 ])
    except KeyError as e:
        print(t)
        raise(e)

# End of function definition
        
# Create an empty dataframe to hold the transaction data
df_raw = pd.DataFrame()

# Call the above function
df_raw = await getTransactions(df_raw, startDate, endDate)
print("Done!")

In [ ]:
# Copy the data to a new variable so we can re-run the data processing without querying the API again.
df = df_raw.copy()

In [ ]:
# How many transactions do we have?
print("Dataframe contains " + str(len(df)) + " transactions.")

## Massage the data into formats ready for plotting

In [ ]:
# Reset the index and sort by date
df = df.sort_values('date', ignore_index=True).reset_index().drop(columns=['index'])

In [ ]:
# Pull some key info to the top level of the dataframe

df['accountName']  = df['account'].apply( lambda x: x.get('displayName') )
df['merchantName'] = df['merchant'].apply( lambda x: x.get('name') )
df['categoryName'] = df['category'].apply( lambda x: x.get('name') )

tag_list = []
for x in df['tags']:
    tgs = ""
    for y in x:
        tgs += y['name'] + ","
    tag_list.append(tgs)

df['Tags'] = tag_list

In [ ]:
# Print the dataFrame keys
df.keys()

In [ ]:
#print(len(df))
#print(len(df[df.date > '2024-02-02']))
df['Date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


In [ ]:
# Remove unneeded columns
# The ones with the data we need are now the Capitalized variants
dfc = df.drop(columns=['date','category','merchant','tags','account'])

# Define the month resample function
def resample(group):
    return group.resample('ME', on='Date').agg({'amount':'sum', 'Date':'last'})

In [ ]:
# Process transactions tagged for exclusion
print("Transactions matching exclusion criteria: " + str(len(dfc[dfc.Tags.str.contains('|'.join(excludeTagsList))])))
# Perform the removal
dfc = dfc[~(dfc.Tags.str.contains('|'.join(excludeTagsList)))]

# Remove any transaction marked for exclusion from reports
dfg = dfc[(dfc.hideFromReports == False)]

# The following line inverts the balances to show expenses as positive numbers
# and refunds as negative numbers.
dfg.loc[:, 'amount'] = dfg.loc[:, 'amount'].apply(lambda x: -x).copy()

# Resample the dataframe into months.  Each month will have each category, which contains the sum of applicable transaction amounts
dfg = dfg.groupby('categoryName').apply(resample, include_groups=False).swaplevel()

# After the resampling, the 'Date' column is now garbage data so lets drop that column.
# The date info that we care about is now part of the index for the dataframe
dfg = dfg.drop(columns=['Date'])

dfg = dfg.reset_index()

In [ ]:
# Remove neutral categories which aren't true expenses
dfg = dfg[~(dfg.categoryName.isin(['Transfer', 'Buy', 'Sell', 'Credit Card Payment']))]
# Remove income categories
dfg = dfg[~(dfg.categoryName.isin(['Paychecks', 'Other Income', 'Dividends & Capital Gains', 'Interest']))]
# Pivot the dataframe to organize the data by category per month
pivot2 = pd.pivot_table(data=dfg, index=['Date'], columns=['categoryName'], values='amount')


# pivot3 keeps the top N categories and groups the rest into 'Other'
numCategories = 10
other_list = list(set(pivot2.keys()) - set(pivot2.sum().sort_values(ascending=False)[:numCategories].keys()) )

pivot3 = pivot2.copy()
pivot3['Other'] = pivot2[other_list].sum(axis=1)
pivot3 = pivot3.drop(columns=other_list)


In [ ]:
# Pivot the dataframe to organize the data by total spend per month
pivotTotal = pd.pivot_table(data=dfg, index=['Date'], columns=['categoryName'], values='amount').copy().sum(axis=1).reset_index()
pivotTotal['Date'] -= datetime.timedelta(days=30)
pivotTotal = pivotTotal.set_index('Date')
pivotTotal.rename(columns={0:'Monthly Spend'}, inplace=True)

# Display the plot
fig = px.bar(pivotTotal, height=600, width=1000, template="plotly_dark")
# Uncomment line below for light mode instead
# fig = px.bar(pivot4, height=600, width=1000)
#pio.renderers.default = 'jupyterlab'
fig.show(renderer="iframe")



##  Plot Monthly Spending Over Specified Time Period

In [ ]:
# Do this with Plotly Express

# Shift the date because otherwise each month is off-by-one
pivot4 = pivot3.reset_index()
pivot4['Date'] -= datetime.timedelta(days=30)
pivot4 = pivot4.set_index('Date')

# Display the plot
fig = px.bar(pivot4, height=600, width=1000, template="plotly_dark")
# Uncomment line below for light mode instead
# fig = px.bar(pivot4, height=600, width=1000)
#pio.renderers.default = 'jupyterlab'
fig.show(renderer="iframe")
#pio.offline.iplot(fig)

## Other Code Examples

### Example: Review monthly spending averages before and after a specific date

In [ ]:
pivot5 = pd.pivot_table(data=dfg, index=['Date'], values='amount', aggfunc='sum')

# Print metrics before specified date
print(pivot5[pivot5.index < '2023-11-01'].describe())
# Print metrics after specified date
print(pivot5[pivot5.index >= '2024-01-01'].describe())

# 2024 Spending total
print("\n2024 Spending Total:")
sumSpendingTotal2024 = pivot5[pivot5.index >= '2024-01-01']['amount'].sum()
print(sumSpendingTotal2024)

# Pivot 3 is required for looking at specific categories
print('\n\n')
# Print metrics for Grocery spend before specified date
print(pivot3[pivot3.index <= '2024-12-30']['Groceries'].describe())
print('\n')
# Print metrics for Grocery spend after specified date
print(pivot3[pivot3.index >= '2024-01-01']['Groceries'].describe())


### Example: Print all transactions from specified category between specified dates

In [ ]:
# Print all transactions from category "Mortgage" between 2024-01-01 and 2024-12-31
print("Total Mortgage spending")
sumMortgage2024 = abs(dfc[(dfc.categoryName == 'Mortgage') & (dfc.Date > '2024-01-01') & (dfc.Date < '2024-12-31') & (dfc.hideFromReports == False)]['amount'].sum())
print(sumMortgage2024)

In [ ]:
# Print all transactions from category "Travel & Vacation" between 2024-01-01 and 2024-03-01
print("Total Property Tax spending")
sumPropertyTax2024 = abs(dfc[(dfc.categoryName == 'Taxes') & (dfc.Date > '2024-01-01') & (dfc.Date < '2024-12-31') & (dfc.hideFromReports == False) & (dfc.merchantName.str.contains("Pty Tax Online Web|eDeposit", regex=True))]['amount'].sum())
print(sumPropertyTax2024)

In [ ]:
print("2024 Spending Total:")
print(sumSpendingTotal2024)
print("2024 Housing Total:")
print(sumMortgage2024 + sumPropertyTax2024)
print("2024 Non-Housing Total:")
print(sumSpendingTotal2024 - (sumMortgage2024 + sumPropertyTax2024))

In [ ]:
# Print all transactions from category "Travel & Vacation" between 2024-01-01 and 2024-03-01
dfc[(dfc.categoryName == 'Travel & Vacation') & (dfc.Date > '2024-01-01') & (dfc.Date < '2024-03-01')]

In [ ]:
# Calculate Costco spend over the past membership year
dfc_tmp = dfc[(dfc.merchantName.str.contains('costco|Costco|COSTCO', regex=True) ) & (dfc.Date > '2023-08-01') & (dfc.Date < '2024-08-01')]
dfc_tmp = dfc_tmp[~dfc_tmp.plaidName.str.contains('gas|Gas|GAS', regex=True)]
print(dfc_tmp.amount.sum())

In [ ]:
# Calculate Whole Foods spend over the past year
dfc_tmp = dfc[(dfc.merchantName.str.contains('whole|Whole|WHOLE', regex=True) ) & (dfc.Date > '2024-01-01') & (dfc.Date < '2024-12-31')]
dfc_tmp = dfc_tmp[(dfc_tmp.merchantName.str.contains('foods|Foods|FOODS', regex=True) ) & (dfc_tmp.Date > '2024-01-01') & (dfc_tmp.Date < '2024-12-31')]
print(dfc_tmp.amount.sum())
print(dfc_tmp.amount.sum()/12)

In [ ]:
# Calculate Clipper spend
dfc_tmp = dfc[(dfc.merchantName.str.contains('clipper|Clipper|CLIPPER', regex=True) ) & (dfc.Date > '2024-01-01') & (dfc.Date < '2024-12-31')]
#dfc_tmp = dfc_tmp[~dfc_tmp.plaidName.str.contains('gas|Gas|GAS', regex=True)]
print(dfc_tmp.amount.sum())